In [1]:
import os
import csv
import sys
from pathlib import Path

sys.path.append(
    os.path.join(
        Path(os.getcwd()).parents[3],
        "src"
    )
)

csv.field_size_limit(sys.maxsize)

131072

In [2]:
import numpy as np
import pandas as pd
from models import AiNet
from representations import NGram
from logging import getLogger
from utils import cosine_distances, euclidean_distances

In [3]:
with open("cleansed_data.csv", "r+", newline="", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data = list(reader)
    header = data.pop(0)

In [4]:
df = pd.DataFrame(data=data, columns=header)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2979 non-null   object
 1   label   2979 non-null   object
dtypes: object(2)
memory usage: 46.7+ KB


In [5]:
df["label"].value_counts().sort_index()

0    1491
1    1488
Name: label, dtype: int64

In [6]:
np.round((df["label"].astype(int).value_counts() / df.shape[0]) * 100, 2).sort_index()

0    50.05
1    49.95
Name: label, dtype: float64

In [20]:
representation = NGram()
features, result = representation.generate_representation(sample["review"].tolist())
result.shape

(5615, 9534)

In [21]:
sys.getsizeof(result)

214133768

In [22]:
sys.getsizeof(result) / (2 ** 30)

0.19942761212587357

In [23]:
cosine_distances(result, result)

array([[0.        , 0.8648496 , 1.        , ..., 1.        , 0.98855263,
        1.        ],
       [0.8648496 , 0.        , 0.9854263 , ..., 1.        , 0.9536495 ,
        0.98339695],
       [1.        , 0.9854263 , 0.        , ..., 0.9806092 , 0.9847667 ,
        1.        ],
       ...,
       [1.        , 1.        , 0.9806092 , ..., 0.        , 0.95522135,
        1.        ],
       [0.98855263, 0.9536495 , 0.9847667 , ..., 0.95522135, 0.        ,
        0.9457799 ],
       [1.        , 0.98339695, 1.        , ..., 1.        , 0.9457799 ,
        0.        ]], dtype=float32)

In [24]:
root_logger = getLogger("root")

In [25]:
aiNet = AiNet(distance_method="cosine", logger=root_logger)

In [26]:
aiNet.fit(
    antigen_population=result, 
    max_iter=15, 
    number_of_antibodies=int(result.shape[0] * 0.10), 
    clone_multiplier=10, 
    no_best_cells_taken_each_selection=5, 
    percent_clones_reselected=0.12, 
    pruning_threshold=0.5
)

In [27]:
aiNet.antibody_population.shape

(130, 9534)

In [28]:
sample.to_csv("sample_cleansed_train_data.csv", encoding="utf-8", index=False, sep=",", quoting=csv.QUOTE_MINIMAL, quotechar='"')

In [30]:
with open("sample_cleansed_train_data.csv", "r+", newline="", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_MINIMAL, quotechar='"')
    data = list(reader)
    header = data.pop(0)

df1 = pd.DataFrame(data=data, columns=header)

In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5615 entries, 0 to 5614
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5615 non-null   object
 1   drugName     5615 non-null   object
 2   condition    5615 non-null   object
 3   review       5615 non-null   object
 4   rating       5615 non-null   object
 5   date         5615 non-null   object
 6   usefulCount  5615 non-null   object
dtypes: object(7)
memory usage: 307.2+ KB


In [32]:
df1["score_class"].value_counts()

8     1952
7     1690
6      951
5      300
9      286
4      170
3       64
10      50
2       36
1        7
0        5
Name: score_class, dtype: int64

In [32]:
(df1["review"].isna().astype(int) + df1["review"].isnull().astype(int)).sum()

0